# Forecast Performance Scorecard

This scorecard provides a systematic, top-down approach to analyze forecast performance and identify areas for model improvement. The process follows a drill-down methodology from high-level business metrics to individual product analysis.

## Scorecard Process Flow:

### 1. **Portfolio Overview** 
Start with the big picture - how is the entire business performing against forecasts? This gives you the overall health of your forecasting system and identifies if there are systematic issues affecting the whole portfolio.

### 2. **Category Performance Analysis**
Drill down into sales categories (Very Low, Low, Alive, Medium, Winning, High Winning) to identify which product segments are driving forecast errors. This helps prioritize which categories need immediate attention.

### 3. **Error Distribution Deep Dive**
For problematic categories, analyze the distribution of forecast errors to understand:
- Are we consistently over-forecasting or under-forecasting?
- What's the magnitude of errors?
- Which products are the biggest contributors to poor performance?

### 4. **Individual Product Investigation**
Focus on the worst-performing products to understand root causes:
- Are there data quality issues?
- Is the product experiencing unusual patterns (promotions, stockouts, seasonality)?
- Are there external factors affecting demand?

### 5. **Actionable Insights Generation**
Based on the analysis, identify specific actions:
- Which products need model retraining?
- What data quality issues need fixing?
- Which categories need different forecasting approaches?
- What external factors should be incorporated into the model?

## How to Use This Scorecard:

1. **Run the complete notebook** to get the full analysis
2. **Start with the business-level view** to understand overall performance
3. **Identify problematic categories** using the error distribution plots
4. **Drill down into specific products** that are driving poor performance
5. **Document findings** walk through lots of skus and identofy dominnat issues then correct. Don't focus on correcting specific edge cases. This will end up messing working cases



# Scorecard

In [ ]:
# =============================================================================
# ALL IMPORTS - CONSOLIDATED IN FIRST CELL
# =============================================================================

# Standard library imports
import sys
import importlib
from importlib import reload

# Third-party imports
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# PySpark imports
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import Window

# Local imports
import configuration as CONFIG
from functions_library.supabase_connection import SupabaseClient
import functions_library.forecast_evaluation as ED

# =============================================================================
# MODULE RELOADING - Ensure latest versions are loaded
# =============================================================================
importlib.reload(CONFIG)
importlib.reload(sys.modules['functions_library.supabase_connection'])
importlib.reload(sys.modules['functions_library.forecast_evaluation'])

# =============================================================================
# SPARK INITIALIZATION
# =============================================================================
spark = SparkSession.builder.appName('ForecastEvaluation').master('local[1]').getOrCreate()
spark.sparkContext.setLogLevel("WARN")

# =============================================================================
# SUPABASE CLIENT INITIALIZATION
# =============================================================================
storage_client = SupabaseClient()

In [ ]:
# Define window functions
w_sku_date = Window.partitionBy("product_id", "date")
w_sku = Window.partitionBy("product_id")


In [185]:
EVALUATION_START_DATE = "2025-10-01"
EVALUATION_END_DATE = "2025-10-07"
RUN_ID = 6

In [186]:
products = storage_client.batch_read_table("products", spark)
prod_day_history = storage_client.download_processed_features(spark)
prod_day_forecast = storage_client.download_forecast_run(RUN_ID, spark)


In [187]:
prod_day_history.count()

185824

In [188]:
prod_day_history.select(F.max("date")).show()

+----------+
| max(date)|
+----------+
|2025-08-04|
+----------+



In [189]:
prod_day_forecast.show()

+--------------------+------------------+------------+--------------+------+---------------+----------+--------------------+
|          product_id|age_sales_category|age_category|sales_category| style|forecast_method|      date|            forecast|
+--------------------+------------------+------------+--------------+------+---------------+----------+--------------------+
|8c91d217-edd1-456...|04| Mature_01| Low|  04| Mature|  02| Very Low|MW1154|        croston|2025-10-01|  0.0303448275862069|
|8c91d217-edd1-456...|04| Mature_01| Low|  04| Mature|  02| Very Low|MW1154|        croston|2025-10-02|  0.0303448275862069|
|8c91d217-edd1-456...|04| Mature_01| Low|  04| Mature|  02| Very Low|MW1154|        croston|2025-10-03|  0.0303448275862069|
|8c91d217-edd1-456...|04| Mature_01| Low|  04| Mature|  02| Very Low|MW1154|        croston|2025-10-04|  0.0303448275862069|
|8c91d217-edd1-456...|04| Mature_01| Low|  04| Mature|  02| Very Low|MW1154|        croston|2025-10-05|  0.0303448275862069|


In [190]:
reload(ED)

<module 'functions_library.forecast_evaluation' from 'c:\\Users\\Cognira\\Desktop\\Aya - Forecast\\functions_library\\forecast_evaluation.py'>

In [191]:
evaluation_data = (prod_day_forecast
                   .join(prod_day_history.select("product_id", "date", "sales_units"), on=["product_id", "date"], how="inner")
                   .groupBy("product_id", "style", "age_sales_category", "age_category", "sales_category", "forecast_method")
                   .agg(F.sum("sales_units").alias("sales_units"),
                        F.sum("forecast").alias("forecast")))
evaluation_data = ED.calculate_bias_and_error_bins(evaluation_data)

In [178]:
validation_data = (prod_day_history
                   .join(prod_day_forecast.select("product_id", "date", "forecast_method", "forecast"), on=["product_id", "date"],how="left"))
validation_data = ED.calculate_bias_and_error_bins(validation_data)

# Business Summary

In [179]:
pdf = ED.plot_business_level(validation_data, EVALUATION_START_DATE=EVALUATION_START_DATE, EVALUATION_END_DATE=EVALUATION_END_DATE)

Evaluation frame (2025-10-01 - 2025-10-07):
  Total Sales: 0
  Total Forecast: 0.0
  Over/Under Forecast: 0.0 units (percent undefined, total sales is zero)


# Overall Evaluation

In [69]:
# Define per-bin good thresholds (understock / overstock)
GOOD_THRESHOLD = {
    "02| Very Low": (5, 10),
    "03| Low": (5, 10),
    "04| Alive": (5, 20),
    "05| Medium": (5, 20),
    "06| Winning": (5, 20), # Bias
    "07| High Winning": (5, 30) # Bias
}

In [70]:
agg=ED.plot_attribute_distribution(evaluation_data, ["sales_category"])


In [180]:
reload(ED)

<module 'functions_library.forecast_evaluation' from 'c:\\Users\\Cognira\\Desktop\\Aya - Forecast\\functions_library\\forecast_evaluation.py'>

In [181]:
ED.plot_accuracy_distribution(evaluation_data, method="bias").show()
ED.plot_accuracy_distribution(evaluation_data, method="error").show()

In [89]:
df = (evaluation_data
    .where(~(F.col("sales_category").isin(["02| Very Low", "03| Low", "07| High Winning"]))))
ED.plot_accuracy_distribution(df, method="bias").show()

# Investigate Sales Categories

In [134]:
SELECTED_SALES_BIN = "07| High Winning" 
METHOD = "bias"

In [133]:
evaluation_data_category = evaluation_data.where(F.col("sales_category") == SELECTED_SALES_BIN)
ED.analyze_category_performance(evaluation_data_category)

,product_count,sales_units,forecast,error
0,3,723,738.336733,15.336733


In [139]:
ED.plot_accuracy_distribution(evaluation_data_category, method=METHOD).show()

# Investigate Products

In [102]:
SELECTED_BIAS_BIN = "22| > +50"
LIMIT = 30

In [103]:
(evaluation_data_category
    .where(F.col("sales_category") == SELECTED_SALES_BIN)
    # .where(F.col("bias_bin").rlike(SELECTED_BIAS_BIN))
    .orderBy(F.desc("error")).limit(LIMIT).toPandas())

,product_id,style,age_sales_category,age_category,sales_category,forecast_method,sales_units,forecast,bias,error,bias_bin,error_bin
0,9c98308b-05cc-44e4-b40b-116d20efd49f,MW1604,04| Mature_03| High,04| Mature,07| High Winning,prophet,205,252.763855,0.232994,47.763855,14| +20 +30,21| +45 +50
1,0e0a1c0d-6a98-4d33-b2dc-66d810894266,MW1341,04| Mature_03| High,04| Mature,07| High Winning,prophet,167,192.752111,0.154204,25.752111,13| +10 +20,17| +25 +30
2,3a1bc094-73ba-47e6-a039-2cd80dd83f4f,MW1537,04| Mature_03| High,04| Mature,07| High Winning,prophet,351,292.820766,-0.165753,-58.179234,10| -20 -10,01| < -50


In [161]:
reload(ED)

<module 'functions_library.forecast_evaluation' from 'c:\\Users\\Cognira\\Desktop\\Aya - Forecast\\functions_library\\forecast_evaluation.py'>

In [162]:
# Example usage
SELECTED_STYLE = "MW1341" 
pdf_product, plot = ED.plot_product_details_daily_short(validation_data, SELECTED_STYLE, products_df=products, EVALUATION_START_DATE=EVALUATION_START_DATE, EVALUATION_END_DATE=EVALUATION_END_DATE)
plot.show()


Product Title: MW1341 -  عباية بليزر بتطريز اسود
Price: 260.87
Status: ACTIVE
